In [1]:
from keras.applications import VGG16

In [2]:
img_rows = 224
img_cols = 224

In [3]:
vgg16_model = VGG16(weights='imagenet',include_top=False,input_shape = (img_rows,img_cols,3))

58892288/58889256 [==============================] - 14s 0us/step


In [4]:
for (i,layer) in enumerate(vgg16_model.layers):
    print(str(i),layer.__class__.__name__,layer.trainable)

0 InputLayer True
1 Conv2D True
2 Conv2D True
3 MaxPooling2D True
4 Conv2D True
5 Conv2D True
6 MaxPooling2D True
7 Conv2D True
8 Conv2D True
9 Conv2D True
10 MaxPooling2D True
11 Conv2D True
12 Conv2D True
13 Conv2D True
14 MaxPooling2D True
15 Conv2D True
16 Conv2D True
17 Conv2D True
18 MaxPooling2D True


In [5]:
for layer in vgg16_model.layers:
    layer.trainable = False

In [6]:
for (i,layer) in enumerate(vgg16_model.layers):
    print(str(i),layer.__class__.__name__,layer.trainable)

0 InputLayer False
1 Conv2D False
2 Conv2D False
3 MaxPooling2D False
4 Conv2D False
5 Conv2D False
6 MaxPooling2D False
7 Conv2D False
8 Conv2D False
9 Conv2D False
10 MaxPooling2D False
11 Conv2D False
12 Conv2D False
13 Conv2D False
14 MaxPooling2D False
15 Conv2D False
16 Conv2D False
17 Conv2D False
18 MaxPooling2D False


In [7]:
def add_top_model(bottom_model,num_classes,D=256):
    
    
    top_model = bottom_model.output
    top_model = Flatten(name='flatten')(top_model)
    top_model = Dense(D,activation='relu')(top_model)
    top_model = Dropout(0.3)(top_model)
    top_model = Dense(num_classes,activation='softmax')(top_model)
    return top_model

In [8]:
from keras.models import Model
from keras.layers import Dense,Dropout,Activation,Flatten
from keras.layers import Conv2D,MaxPooling2D,ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.models import Sequential

In [10]:
num_classes = 17
FC_head = add_top_model(vgg16_model,num_classes)

model = Model(inputs=vgg16_model.input,outputs = FC_head)
print(model.summary())

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)      

In [11]:
from keras.preprocessing.image import ImageDataGenerator

In [12]:
train_data_dir = './17_flowers/train/'
validation_data_dir = './17_flowers/validation/'

train_generator = ImageDataGenerator(rescale=1./255,rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,horizontal_flip=True,fill_mode='nearest')

In [13]:
validation_generator = ImageDataGenerator(rescale=1./255)

In [14]:
train_batch_size = 16
validation_batch_size = 10

In [15]:
train_datagen = train_generator.flow_from_directory(train_data_dir,target_size=(img_rows,img_cols),batch_size=train_batch_size,class_mode='categorical')

Found 1190 images belonging to 17 classes.


In [16]:
validation_datagen = validation_generator.flow_from_directory(validation_data_dir,target_size=(img_rows,img_cols),batch_size=validation_batch_size,class_mode='categorical',shuffle=False)

Found 170 images belonging to 17 classes.


In [17]:
nb_train_images = 1190
nb_validation_images = 170

## Training the top model

In [18]:
from keras.optimizers import RMSprop

In [19]:
from keras.callbacks import ModelCheckpoint,EarlyStopping

In [20]:
checkpoint = ModelCheckpoint('Flowers_Transfer_Learning.h5',monitor='val_loss',mode='min',save_best_only=True,verbose= True)

In [22]:
early_stopping = EarlyStopping(monitor='val_loss',min_delta=0,patience=3,restore_best_weights=True,verbose=1)

In [23]:
callbacks = [early_stopping,checkpoint]

In [24]:
model.compile(loss='categorical_crossentropy',optimizer = RMSprop(lr=0.001),metrics=['accuracy'])

In [25]:
epochs = 3
batch_size = 16

In [26]:
history = model.fit_generator(train_datagen,steps_per_epoch=nb_train_images//batch_size,epochs = epochs,validation_data=validation_datagen,callbacks=callbacks,validation_steps=nb_validation_images//batch_size)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/3
74/74 [==============================] - ETA: 0s - loss: 3.8180 - accuracy: 0.2172
Epoch 00001: val_loss improved from inf to 1.85216, saving model to Flowers_Transfer_Learning.h5
74/74 [==============================] - 275s 4s/step - loss: 3.8180 - accuracy: 0.2172 - val_loss: 1.8522 - val_accuracy: 0.4000
Epoch 2/3
74/74 [==============================] - ETA: 0s - loss: 1.9279 - accuracy: 0.3876
Epoch 00002: val_loss improved from 1.85216 to 1.16654, saving model to Flowers_Transfer_Learning.h5
74/74 [==============================] - 259s 4s/step - loss: 1.9279 - accuracy: 0.3876 - val_loss: 1.1665 - val_accuracy: 0.7000
Epoch 3/3
74/74 [==============================] - ETA: 0s - loss: 1.5841 - accuracy: 0.5119
Epoch 00003: val_loss improved from 1.16654 to 1.03763, saving model to Flowers_Transfer_Learning.h5
74/74 [==============================] - 304s 4s/step - loss: 1.5841 - accuracy: 0.511

In [27]:
model.save('Flowers_Classifier.h5')